In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shadowdealer","key":"c14e46bf79b604e2e6cc9c01176f95a8"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c case3-datasaur-photo

 98% 0.98G/0.99G [00:10<00:00, 144MB/s]
100% 0.99G/0.99G [00:10<00:00, 102MB/s]


In [ ]:
!unzip case3-datasaur-photo.zip -d data

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70090330.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70090667.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70113673.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70116580.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70132146.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70140824.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70159945.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70176903.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop/70181649.jpeg  
  inflating: data/techosmotr/techosmotr/train/fictivniye(fi

In [ ]:
import os
import random
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
import os
import shutil
import random

# Constants for directory names
BASE_PATH = "/content/data/techosmotr/techosmotr"
TRAIN_PATH = os.path.join(BASE_PATH, "train")
FICTITIOUS_PATH = os.path.join(TRAIN_PATH, "fictivniye(fictitious)")
CORRECT_PATH = os.path.join(TRAIN_PATH, "pravilniye(correct)")
COMBINED_FICTIVE_PATH = os.path.join(TRAIN_PATH, "fictivniye_combined")
CORRECT_SUBFOLDER = os.path.join(CORRECT_PATH, "0-correct")
TRAINING_SET_PATH = os.path.join(TRAIN_PATH, "training_set")
VALIDATION_SET_PATH = os.path.join(TRAIN_PATH, "validation_set")

FICTIVE_SUBFOLDERS = [
    "1-not-on-the-brake-stand",
    "2-from-the-screen",
    "3-from-the-screen+photoshop",
    "4-photoshop"
]

# Function to create a directory if it doesn't exist
def create_directory(directory):
    """Create a directory if it does not exist."""
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to copy files from source to destination
def copy_files(source_folder, destination_folder):
    """Copy files from source folder to destination folder."""
    for file in os.listdir(source_folder):
        src_file = os.path.join(source_folder, file)
        dst_file = os.path.join(destination_folder, file)
        shutil.copy2(src_file, dst_file)

# Function to split images into training and validation sets
def split_images(source_folder, train_folder, val_folder, split_ratio=0.8):
    """Split images from source folder into training and validation sets."""
    images = os.listdir(source_folder)
    random.shuffle(images)
    train_count = int(len(images) * split_ratio)
    train_images = images[:train_count]
    val_images = images[train_count:]
    
    for image in train_images:
        shutil.copy2(os.path.join(source_folder, image), os.path.join(train_folder, image))
    for image in val_images:
        shutil.copy2(os.path.join(source_folder, image), os.path.join(val_folder, image))

# Create directories
create_directory(COMBINED_FICTIVE_PATH)
for directory in [TRAINING_SET_PATH, VALIDATION_SET_PATH]:
    for sub_dir in ["fictivniye", "pravilniye"]:
        create_directory(os.path.join(directory, sub_dir))

# Combine all fictitious images into one directory
for subfolder in FICTIVE_SUBFOLDERS:
    subfolder_path = os.path.join(FICTITIOUS_PATH, subfolder)
    copy_files(subfolder_path, COMBINED_FICTIVE_PATH)

# Split the combined fictitious and correct images into training and validation sets
split_images(COMBINED_FICTIVE_PATH, os.path.join(TRAINING_SET_PATH, "fictivniye"), os.path.join(VALIDATION_SET_PATH, "fictivniye"))
split_images(CORRECT_SUBFOLDER, os.path.join(TRAINING_SET_PATH, "pravilniye"), os.path.join(VALIDATION_SET_PATH, "pravilniye"))


In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 640
EPOCHS = 10
LEARNING_RATE = 0.001
PATIENCE = 5
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

In [ ]:
train_dataset = datasets.ImageFolder(TRAINING_SET_PATH, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(VALIDATION_SET_PATH, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Load and modify the ResNet50 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True)

# Freeze all layers except the last ones
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 312MB/s]


In [ ]:
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 2)
)

In [ ]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

In [ ]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)

In [ ]:
import torch
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

def perform_training_epoch(epoch_num, model, data_loader, device, optimizer, loss_fn):
    """
    Perform a single training epoch.
    
    Args:
    - epoch_num (int): The current epoch number.
    - model (torch.nn.Module): The neural network model.
    - data_loader (DataLoader): DataLoader for the training data.
    - device (torch.device): The device to run the model on.
    - optimizer (torch.optim.Optimizer): The optimizer.
    - loss_fn (callable): The loss function.

    Returns:
    - float: The average loss of the epoch.
    """
    model.train()
    total_loss = 0.0

    for batch_data, batch_target in tqdm(data_loader, desc=f"Epoch {epoch_num}"):
        batch_data, batch_target = batch_data.to(device), batch_target.to(device)
        optimizer.zero_grad()
        predictions = model(batch_data)
        loss = loss_fn(predictions, batch_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch_num}: Avg Training Loss: {avg_loss}")
    return avg_loss

def perform_validation_step(model, data_loader, device, loss_fn):
    """
    Perform a validation step.

    Args:
    - model (torch.nn.Module): The neural network model.
    - data_loader (DataLoader): DataLoader for the validation data.
    - device (torch.device): The device to run the model on.
    - loss_fn (callable): The loss function.

    Returns:
    - float: The F1 score for the validation data.
    """
    model.eval()
    total_val_loss = 0.0
    all_predicted_labels = []
    all_true_labels = []

    with torch.no_grad():
        for batch_data, batch_target in tqdm(data_loader, desc="Validation"):
            batch_data, batch_target = batch_data.to(device), batch_target.to(device)
            predictions = model(batch_data)
            loss = loss_fn(predictions, batch_target)
            total_val_loss += loss.item()

            predicted_labels = predictions.argmax(dim=1)
            all_predicted_labels.extend(predicted_labels.cpu().numpy())
            all_true_labels.extend(batch_target.cpu().numpy())

    avg_val_loss = total_val_loss / len(data_loader)
    f1_metric = calculate_metrics(all_true_labels, all_predicted_labels)
    print(f"Validation Loss: {avg_val_loss} | F1 Score: {f1_metric}")
    return f1_metric

def calculate_metrics(true_labels, predicted_labels):
    """
    Calculate metrics such as F1 score and accuracy.

    Args:
    - true_labels (list): The true labels.
    - predicted_labels (list): The predicted labels.

    Returns:
    - float: The F1 score.
    """
    f1_metric = f1_score(true_labels, predicted_labels, average='macro')
    accuracy_metric = accuracy_score(true_labels, predicted_labels)
    print(f"F1 Score: {f1_metric} | Accuracy: {accuracy_metric}")
    return f1_metric


In [ ]:
best_f1 = 0.0
counter = 0
for epoch in range(1, EPOCHS+1):
    train(epoch)
    current_f1 = validate()
    scheduler.step(current_f1)

    # Save the best model
    if current_f1 > best_f1:
        best_f1 = current_f1
        torch.save(model.state_dict(), 'best_model.pth')
        counter = 0
    else:
        counter += 1
        if counter >= PATIENCE:
            print("Early stopping triggered!")
            break

print("Training Completed!")

Epoch 1: 100%|██████████| 165/165 [04:32<00:00,  1.65s/it]


Epoch 1: Training Loss: 0.18423002318225123


Validating: 100%|██████████| 42/42 [00:55<00:00,  1.33s/it]


Validation Loss: 0.19031659025876296 | F1 Score: 0.9318026802891964 | Accuracy: 0.9415781487101669


Epoch 2: 100%|██████████| 165/165 [04:36<00:00,  1.67s/it]


...

Validation Loss: 0.16708684630048948 | F1 Score: 0.959120981074878 | Accuracy: 0.9650986342943855


Epoch 10: 100%|██████████| 165/165 [04:36<00:00,  1.68s/it]


Epoch 10: Training Loss: 0.011616550271102108


Validating: 100%|██████████| 42/42 [00:55<00:00,  1.33s/it]

Validation Loss: 0.18404897830554373 | F1 Score: 0.9538515406162464 | Accuracy: 0.960546282245827
Epoch 00010: reducing learning rate of group 0 to 1.0000e-05.
Training Completed!


In [ ]:
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd

# Global Constants
TEST_PATH = "/content/data/techosmotr/techosmotr/test"
IMG_SIZE = 224  # Example size, adjust as necessary
BATCH_SIZE = 32  # Example batch size, adjust as necessary

# Custom dataset for test images
class TestDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.image_files = [f for f in os.listdir(path) if f.endswith('.jpeg')]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.path, self.image_files[idx])
        try:
            image = Image.open(img_name)
        except IOError:
            print(f"Error opening image {img_name}")
            return None, None

        if self.transform:
            image = self.transform(image)

        return image, self.image_files[idx]

def get_test_transform():
    """ Returns the transformation applied to test images. """
    return transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

def test_model(model, test_loader, device):
    """ Perform inference on the test dataset using the provided model. """
    model.eval()
    model.to(device)
    all_preds = []
    all_files = []

    for inputs, filenames in test_loader:
        if inputs is None:
            continue
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            all_preds.extend(preds.cpu().numpy())
            all_files.extend(filenames)

    return all_files, all_preds

def save_predictions_to_csv(filenames, predictions):
    """ Save the test predictions to a CSV file. """
    results = {
        'file_index': [f.split('.jpeg')[0] for f in filenames],
        'class': [int(pred[0]) for pred in predictions]
    }
    result_df = pd.DataFrame(results)
    result_df.to_csv("submission.csv", index=False)

# Main execution
test_transform = get_test_transform()
test_dataset = TestDataset(TEST_PATH, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Assuming 'model' and 'device' are defined and initialized elsewhere
test_files, test_predictions = test_model(model, test_loader, device)
save_predictions_to_csv(test_files, test_predictions)
